# Known-Artist Live Song ID: A Hashprint Approach

This notebook is a python implementation of the [original matlab version] (http://pages.hmc.edu/ttsai/assets/livesongid.tar.gz). The pipeline consists of the following modules:

1. Preprocessing with Constant-Q Transform (CQT)
2. Learning PCA filters
3. Obtaining hashprint representation by applying PCA filters to preprocessed CQT matrix
4. Generating database with the hashprint of pitch-shifted CQT matrix of reference tracks
5. Matching query against database

In [1]:
GIT_DIR = '/home/zhwang/workspace/live-song-id/'
%cd $GIT_DIR

/home/zhwang/workspace/live-song-id


## Data Preprocessing

For a given artist, we first preprocess the list of reference tracks by taking CQT transform for each audio file.

In [66]:
from preprocess import *
import os
from utils import pitch_shift_CQT

In [ ]:
# audio_path - where audio files are stored
# list_dir - where list of softlinks are stored
# cqt_dir - where results will be stored
audio_path = '/home/nbanerjee/SoftLinks/'
list_dir = audio_path + '/Lists/'
cqt_dir = '/home/zhwang/ttemp/livesong_results/' # change this
artist = 'taylorswift'
out_dir = os.path.join(cqt_dir, artist+'_out')
file_paths = get_allpaths(artist, list_dir)

%cd $cqt_dir
%mkdir $out_dir
%cd $out_dir

In [ ]:
f = open(os.path.join(out_dir, artist + '_cqtList.txt'), 'w')
for cur_file in file_paths:
    print('==> Taking CQT of %s'%cur_file)
    y, sr = librosa.load(audio_path + cur_file + '.wav')
    Q = librosa.cqt(y, sr=sr, fmin=130.81, n_bins=121, bins_per_octave=24)
    logQ = preprocess(Q, 3)
    cur_file_path = os.path.join(os.getcwd(), cur_file + '.dat')
    np.savetxt(cur_file_path, logQ)
    f.write(cur_file_path + '\n')
f.close()


## Learning PCA Filters

In [4]:
%cd $GIT_DIR
from PCA import *

/home/zhwang/workspace/live-song-id


Next, we generate the filters for the hashprint representation by taking PCA on the covariance matrix for all reference songs for this artist. First, we compute the covariance matrix:

In [ ]:
f = open(os.path.join(out_dir, artist + '_cqtList.txt'), 'r')
num_features = 64
nbins = 121
m = 20 # number of frames

In [ ]:
accum_cov = np.zeros((nbins * m, nbins * m))
count = 0

for line in f:
    print('==> Computing covariance matrix for %s'%os.path.basename(line)[:-1])
    Q = np.loadtxt(line[:-1])
    A = getTDE(Q)
    if A.shape[1] > 1:
        accum_cov += np.cov(A.T)
    count += 1

In [55]:
f.close()

Next, we compute PCA from these covariance matrices.

In [21]:
evals, evecs = LA.eig(accum_cov / count)
ind = np.argsort(-evals)
evecs = (evecs[:, ind])[:, :num_features]
np.savetxt(os.path.join(out_dir, artist + '_model.dat'), evecs)

## Applying Filters to CQT Matrix to generate the Database

In [62]:
evecs = np.loadtxt(os.path.join(out_dir, artist + '_model.dat')).T
from model import *
pca_matrix = np.array([vec.reshape((m, -1)).T for vec in evecs])
delta = 16
max_pitch_shift = 4

In [68]:
f = open(os.path.join(out_dir, artist + '_cqtList.txt'), 'r')
for line in f:
    print('==> Generating database for %s'%os.path.basename(line)[:-1])
    Q = np.loadtxt(line[:-1])
    pitch_shift_Qs = np.empty((2 * max_pitch_shift + 1, ) + Q.shape)
    pitch_shift_Qs[0, :, :] = Q
    for i in range(1, max_pitch_shift + 1):
        pitch_shift_Qs[i, :, :] = pitch_shift_CQT(Q, i)
    for i in range(1, max_pitch_shift + 1):
        pitch_shift_Qs[i + max_pitch_shift, :, :] = pitch_shift_CQT(Q, -i)
    print(pitch_shift_Qs.shape)
    
    conv_1d_net = build_model(pca_matrix, Q.shape, 9, delta=delta)
    fpseqs = run_model(conv_1d_net, pitch_shift_Qs, 9)
    break
#     print(origfpseq.shape)
#     break
        

==> Generating database for taylorswift_ref1.dat
(9, 121, 2809)


ValueError: The `kernel_size` argument must be a tuple of 1 integers. Received: (121, 20)

## Matching Query Against Database